In [1]:
import pandas as pd
import json

In [2]:
def clean_element_emballage(elements, materials):
    new_materials = [item.replace('\n', '').strip() for item in materials]
    new_materials = [item for item in new_materials if item]

    new_elements = [item.replace('\n', '').strip() for item in elements]
    new_elements = [item for item in new_elements if item]

    return new_elements, new_materials

In [3]:
def combine_elements_and_materials(elements, materials):
    combined_list = []
    
    if len(elements) == len(materials):
        # Cuando las longitudes son iguales
        for element, material in zip(elements, materials):
            combined_string = f"{material} : {element}"
            combined_list.append(combined_string)
    elif len(materials) * 2 == len(elements):
        # Cuando la longitud de 'elements' es el doble de 'materials'
        for i in range(len(materials)):
            quantity = elements[2 * i]
            weight = elements[2 * i + 1]
            material = materials[i]
            combined_string = f"{quantity} {material} : {weight}"
            combined_list.append(combined_string)
    else:
        print("Las listas no tienen una relación 1:1 o 1:2.")
        print(elements)
        print(materials)
    
    return combined_list

In [4]:
def update_json_file(file_path):
    # Cargar el JSON
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Modificar cada producto en el JSON
    for product in data:
        elements = product['environment_data']['packaging']['element_emballage']['element']
        materials = product['environment_data']['packaging']['element_emballage']['material']
        
        # Limpiar y combinar los elementos y materiales
        cleaned_elements, cleaned_materials = clean_element_emballage(elements, materials)
        combined_packaging = combine_elements_and_materials(cleaned_elements, cleaned_materials)
        
        # Sobreescribir 'element_emballage' con la lista combinada
        product['environment_data']['packaging']['element_emballage'] = combined_packaging
    
    # Guardar el JSON modificado con codificación utf-8
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)



In [5]:
import os

In [6]:
path = 'open_food/data/'

#for file in os.listdir('open_food/data'):
#    update_json_file(path + file)

In [7]:
def concatenate_json_files(input_files, output_file):
    combined_data = []

    # Leer y combinar el contenido de cada archivo JSON
    for file_path in input_files:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            combined_data.extend(data)  # Asume que cada archivo contiene una lista de elementos

    # Guardar los datos combinados en un nuevo archivo JSON
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(combined_data, file, ensure_ascii=False, indent=4)

In [8]:
input_files = ['open_food/data/'+file for file in os.listdir('open_food/data')]
output_file = 'open_food/data/total_open_food.json'

In [9]:
concatenate_json_files(input_files, output_file)

In [10]:
def remove_duplicates(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        products = json.load(file)

    seen_barcodes = set()
    unique_products = []

    for product in products:
        bar_code = product['basic_data'].get('bar_code')
        if bar_code and bar_code not in seen_barcodes:
            unique_products.append(product)
            seen_barcodes.add(bar_code)

    return unique_products

def save_to_file(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [11]:
input_file_path = 'open_food/data/total_open_food.json'
output_file_path = 'total_unique_products.json'

unique_products = remove_duplicates(input_file_path)
save_to_file(unique_products, output_file_path)